In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
  temperature=0.1, 
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
  ]
)

In [2]:
#랭체인에는 5갖 정도의 메모리가 있는데, 모드 저장 방식도 다르고, 각자만의 장단점이 있다.
#챗복에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없고, 대화를 이해할 수 있는 능력이 없다.
#openai 에서 default 로 제공하는 것은 메모리를 지원하지 않아 stateless 하다.
#모델에게 무언가를 보내면 응답한 후 모든 대화 내용을 저장하지 않아 잊는다.
#chatGPT에는 메모리가 탑재되어있어 실제로 어떤 사람과 이야기하고 있다는 느낌을 들게 한다.

#Conversation Buffer memory - 단순히 이전 대화 내용 전체를 저장하는 메모리
#단점은 대화 내용이 길어질수록 메모리도 계속 커지기 때문에 비효율적이다.
#모델 자체에는 메모리가 없기 때문에 모델에 요청을 보낼 때 이전 대화 기록 전체를 같이 보내야한다.
#대화가 길어질수록 메모리에 보내야될 대화가 길어진다.

from langchain.memory import ConversationBufferMemory

#text completion을 할때 유용하다.예측을하고, 텍스트를 자동 완성 하고 싶을 때
#챗모델과 작업을 하려면 스트링이 아닌 ai 메세지와 human메세지가 모두 필요하다.
#메모리 종류와 무관하게 api는 다 같다.(return_message, save_context, load_memory...등 을 갖고 있음)
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"hi"},{"output":"how are you"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='how are you')]}

In [4]:
#대화의 특정 부분만을 저장하는 메모리
#가장 최근의 5개의 메세지를 저장한다고 했을 떄, 6번째 메세지가 추가되었을 때 가장 오래된 메세지는 버려지는 방식
#메모리를 특정 크기로 유지할 수 있는것이 장점
#단잠은 챗봇이 전체 대화가 아닌 최근 대화에만 집중한다.
from langchain.memory import ConversationBufferWindowMemory

#k-how many interaction you want to remember
memory = ConversationBufferWindowMemory(
  return_messages=True, 
  k=4
)

def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})


add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [5]:
add_message(5,5)

# 첫번째는 사라짐
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [6]:
#메세지를 오는 그대로 저장하는것이 아니라 요약해서 저장한다.
#conversationBuffferMemory를 사용하면 대화가 진행될수록 저장된 모든 메세지가 많이지면서 연결된다.
#초반에는 이전보다 더 많은 토큰과 저장공간을 차지하게 되지만, summary를 사용하면 대화가 쌓일수록 요약되어 토큰의 양도 줄어들고 훨씬 나은 방법이 된다.
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

#메모리를 실행하는데 비용이 듬
memory= ConversationSummaryMemory(llm=llm)

def get_history():
  return memory.load_memory_variables({})

add_message("hi I'm seoyul, I live in south korea", "Wow that is so cool")

In [7]:
add_message("south korea is so pretty", "I wish i could go!!!")

get_history()

{'history': 'Seoyul from South Korea expresses admiration for their country. The AI responds by expressing a desire to visit South Korea.'}

In [32]:
#conversation summary memory + conversation buffer memory의 결합
#메모리에 보내온 베세지의 수를 저장
#Limit 에 도달하는 순간 이전에 일어났던 대화들을 잊어버리는 것이 아닌 요약하게된다.
#가장 최근의 상호작용을 계속 추적한다.
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(temperature=0.1)

llm = ConversationSummaryBufferMemory(
  llm = llm,
  #메세지가 요약되기 전 가능한 토큰 수의 최대값
  max_token_limit=10,
  return_messages=True
)


add_message("south korea is so pretty", "I wish i could go!!!")

get_history()


In [13]:
#대화중의 엔티티의 knowledge graph를 만든다.
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
  llm=llm,
  return_messages=True
)

add_message("Hi I'm Zoe, I live in south korea", "wow that is so cool!")

In [14]:
#요약하지만 대화에서 entity를 뽑아낸다
memory.load_memory_variables({"input":"who is Zoe"})

{'history': [SystemMessage(content='On Zoe: Zoe lives in South Korea.')]}

In [15]:
add_message("Zoe likes coffee", "wow that is so cool!")
memory.load_memory_variables({"input":"What does Zoe like"})

{'history': [SystemMessage(content='On Zoe: Zoe lives in South Korea. Zoe likes coffee.')]}

In [33]:
#llm chain - off the shelf(일반적인 목적을 가진) chain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  #템플릿 안에 컨텐츠를 넣으라는 의미로 템플릿 안에 메모리가 히스토리를 저장하도록 한 곳을 적어줌
  #메모리는 템플릿 안에 대화 기록을 저장하는 공간을 찾고, ㅏㅈ동으로 메모리의 기록을 넣는다.
  memory_key="chat_history"
)

template = """
  You are a helpful AI talking to a human.
  
  {chat_history}
  Human:{question}
  You:
"""

chain = LLMChain(
  llm=llm,
  memory=memory,
  prompt=PromptTemplate.from_template(template),
  #chain을 실행했을 때 프롬프트 로그들을 확인할 수 있다.
  verbose=True
)

chain.predict(question="My name is Zoe")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful AI talking to a human.
  
  
  Human:My name is Zoe
  You:


> Finished chain.


'Hello Zoe! How can I assist you today?'

In [34]:
chain.predict(question="I live in seoul")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful AI talking to a human.
  
  Human: My name is Zoe
AI: Hello Zoe! How can I assist you today?
  Human:I live in seoul
  You:


> Finished chain.


"That's great to know! Is there anything specific you would like assistance with regarding Seoul?"

In [35]:
chain.predict(question="What is my name?")

# 우리가 원하는 방식으로 프롬프트에게 대화 기록을 추가해줘야함



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful AI talking to a human.
  
  Human: My name is Zoe
AI: Hello Zoe! How can I assist you today?
Human: I live in seoul
AI: That's great to know! Is there anything specific you would like assistance with regarding Seoul?
  Human:What is my name?
  You:


> Finished chain.


'Your name is Zoe.'

In [36]:
#문자열형태, message 형태
#메모리 옵션에 return_message=True 를 사용하면 다음 명령어 사용시 메세지 형태로 리턴된다.
#memory.load_memory_variables({})
from langchain.prompts import MessagesPlaceholder,ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  memory_key="chat_history",
  return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  #누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한 없는 양의 메세지를 가질 수 있다.
  # 다음을 사용하면 메세지가 얼마나 누구에게로부터 왔는지 모르지만 memory class로 대체될것이다.
  MessagesPlaceholder(variable_name="chat_history"),
  ("human", "{question}")
])

def load_memory():
  return memory.load_memory_variables({})["chat_history"]

#RunnablePassthrough 를 사용하면 load_memory 함수의 결과값을 chat_history input으로 넣어주라고 알려줄 수 있다.
#프롬프트가 format 됮기 전에 함수를 실행시키는걸 허락해준다. 원하는 변수를 넣을 수 있고, 그 변수는 prompt에 주어지게 된다.
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

#chain을 invoke 할때마다 chat history를 더해줘야함
chain.invoke({
  "chat_history": load_memory(),
  "question":"My name is Seoyul"
})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Zoe

> Finished chain.


'Hello Zoe! How can I assist you today?'

In [50]:
from langchain.prompts import MessagesPlaceholder,ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="chat_history"),
  ("human", "{question}")
])

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
  result = chain.invoke(
    {
      "question":question
    }
  )
  #메모리를 수동으로 관리하고 있기 때문에 사용자와 기계간의 인터렉션을 메모리에 저장해야한다.
  #save_context - input과 output을 메모리에 저장한다.
  memory.save_context({"input": question}, {"output": result.content})
  print(result)


In [51]:
invoke_chain("My name is seoyul")

content='Hello Seoyul! How can I assist you today?'


In [52]:
invoke_chain("What is my name?")

content='Your name is Seoyul.'
